In [1]:
from gcs import path_handler as ph
import gcs
import os 
import h5py
import stream_analysis as sa
import numpy as np 
from astropy.io import fits

import plotly.graph_objects as go

In [2]:
def get_masses_and_radii(GCnames,montecarlokey):
    Masses,rh_mes,_,_,_,_,_,_=gcs.extractors.MonteCarloObservables.extract_all_GC_observables(GCnames,montecarlokey)
    rs = np.array([gcs.misc.half_mass_to_plummer(x).value for x in rh_mes])
    Masses = np.array([x.value for x in Masses])    
    return Masses,rs

In [3]:
def extract_foorb_data(infilename):
    with h5py.File(infilename, "r") as foorb:
        ax=foorb['ax'][()]
        ay=foorb['ay'][()]
        az=foorb['az'][()]
        tau=foorb['tau'][()]
        time=foorb['time'][()]
        tau_indexing=foorb['tau_indexing'][()]
        time_indexing=foorb['time_indexing'][()]
        magnitude = np.sqrt(ax**2 + ay**2 + az**2)
    return ax,ay,az,tau,time,tau_indexing,time_indexing,magnitude


In [4]:
def get_perturbing_orbits(GCnames, MWpotential, montecarlokey):
    ts,xs,ys,zs,vxs,vys,vzs=gcs.extractors.GCOrbits.extract_orbits_from_all_GCS(GCnames, MWpotential, montecarlokey) # type: ignore
    today_index = np.argmin(np.abs(ts))
    ts=ts[0:today_index]
    xs,ys,zs=xs[:,0:today_index],ys[:,0:today_index],zs[:,0:today_index]
    vxs,vys,vzs=vxs[:,0:today_index],vys[:,0:today_index],vzs[:,0:today_index]
    return ts,xs,ys,zs,vxs,vys,vzs

In [5]:
def get_perturber_names(GCname):
    path_to_cluster_data="/home/sferrone/tstrippy/tstrippy/data/2023-03-28-merged.fits"
    with fits.open(path_to_cluster_data) as myfits:
        myGCnames=myfits[1].data['Cluster']
    GCnames = [str(myGCnames[i]) for i in range(len(myGCnames))]
    GCnames.pop(GCnames.index(GCname))
    return GCnames

In [6]:
def extraxt_dau_density(path_tau):
    with h5py.File(path_tau,'r') as myfile:
        time_stamps = myfile['time_stamps'][:]
        tau_centers = myfile['tau_centers'][:]
        tau_counts  = myfile['tau_counts'][:]
    return time_stamps,tau_centers,tau_counts

In [57]:

def prepare_proxy_stream(tau,hostorbit,accels,my_tau_left,my_tau_right,my_time_indexing,tau_indexing):
    """
    The convience functions down samples the FOORB to only be the lenght permitted by the tau density map.
    The function assums that the host orbit and the FOORB are indexed to the same time array.
    The function also requires that the accels are already downsampled to the time in question

    """


    # unpack the host orbit
    xH,yH,zH=hostorbit
    # unpack the accelerations
    ax,ay,az,magnitude=accels
    assert(len(ax.shape)==1)
    assert(len(ay.shape)==1)
    assert(len(az.shape)==1)

    ### THE hostorbit and accels are already indexed to the time and tau arrays
    # example 
    # xH[my_time_indexing] is the x position of the host at the time of the stream
    
    blank_dexes=np.arange(0,len(tau),1)
    leftdex=np.argmin(np.abs(tau - my_tau_left))
    rightdex=np.argmin(np.abs(tau - my_tau_right))
    cond1 = blank_dexes >= leftdex
    cond2 = blank_dexes <= rightdex
    cond = np.logical_and(cond1, cond2)
    # extract the stream-proxy data
    x_stream=xH[my_time_indexing + tau_indexing[cond]]
    y_stream=yH[my_time_indexing + tau_indexing[cond]]
    z_stream=zH[my_time_indexing + tau_indexing[cond]]

    # get the force on the stream-proxy
    ax_sub=ax[cond]
    ay_sub=ay[cond]
    az_sub=az[cond]
    magnitude_sub=magnitude[cond]
    # normalize the force
    return x_stream,y_stream,z_stream,ax_sub,ay_sub,az_sub,magnitude_sub


In [58]:
def colored_objects(backgroundcolor="black"):
    if backgroundcolor == "black":
        textcolor = "white"
        linecolor = "gray"
        perturbercolor = "white"
    else:
        textcolor = "black"
        linecolor = "gray"
        perturbercolor = "black"
    return textcolor,linecolor,perturbercolor

In [59]:
# data parameters
GCname              =   "Pal5"
mwpotential         =   "pouliasis2017pii-GCNBody"
montecarlokey       =   "monte-carlo-000"
NP                  =   int(1e5)
internal_dynamics   =   "isotropic-plummer"

In [60]:
# hyperparameters
threshold = 50
limits = 15
camera_distance=1.5
backgroundcolor = "black"


In [61]:
# paths
path_tau        =   ph.tauDensityMaps(GCname,mwpotential,montecarlokey,NP,internal_dynamics)
path_FOORB      =   ph.ForceOnOrbit(GCname, mwpotential, montecarlokey, )
path_host_orb   =   ph.GC_orbits(mwpotential, GCname)

In [62]:
# open the perturber data
GCnames                 =   get_perturber_names(GCname)
ts,xs,ys,zs,vxs,vys,vzs =   get_perturbing_orbits(GCnames, mwpotential, montecarlokey)
Masses,rs               =   get_masses_and_radii(GCnames,montecarlokey)

In [63]:
# get the host orbit 
tH,xH,yH,zH,vxH,vyH,vzH     =   gcs.extractors.GCOrbits.extract_whole_orbit(path_host_orb,montecarlokey=montecarlokey)

In [64]:
thetaH=np.arctan2(yH,xH)
unwraptheta=np.unwrap(thetaH)
meanThetaDot = (unwraptheta[-1]-unwraptheta[0])/(tH[-1]-tH[0])

In [65]:
# open the FORCE ON ORBIT data
ax,ay,az,tau,time,tau_indexing,time_indexing,magnitude = extract_foorb_data(path_FOORB)

In [66]:
# extract 1D stream density profile and get the desired length
time_stamps,tau_centers,tau_counts  =   extraxt_dau_density(path_tau)
left_indexes,right_indexes          =   sa.streamLength.get_envelop_indexes(tau_counts,threshold)
tau_left,tau_right                  =   sa.streamLength.tau_envelopes(tau_centers,left_indexes,right_indexes)

In [67]:
# pre plot prep
textcolor,linecolor,perturbercolor = colored_objects(backgroundcolor)

In [68]:
sizes = np.log10(Masses)
sizes = np.sqrt(Masses/np.max(Masses)*100)

### Sub sample data at a given timestep

In [69]:
blank_dexes=np.arange(0,tau_indexing.shape[0],1)

In [70]:
xaxisline = go.Scatter3d(x=[-limits, limits], y=[0, 0], z=[0, 0], marker=dict(color=linecolor, size=1),showlegend=False)
yaxisline = go.Scatter3d(x=[0, 0], y=[-limits, limits], z=[0, 0], marker=dict(color=linecolor, size=1),showlegend=False)
zaxisline = go.Scatter3d(x=[0, 0], y=[0, 0], z=[-limits, limits], marker=dict(color=linecolor, size=1),showlegend=False)





xaxis=dict(
        title='',
        range=[-limits, limits],  # Set the x-axis limits
        showbackground=False,  # Remove the background pane
        showgrid=False,  # Remove the grid lines
        zeroline=False,  # Remove the zero line
        showticklabels=False,
        )
yaxis=dict(
        title='',
        range=[-limits, limits],  # Set the y-axis limits
        showbackground=False,  # Remove the background pane
        showgrid=False,  # Remove the grid lines
        zeroline=False,  # Remove the zero line
        showticklabels=False,
        )
zaxis=dict(
        title='',
        range=[-limits, limits],  # Set the z-axis limits
        showbackground=False,  # Remove the background pane
        showgrid=False,  # Remove the grid lines
        zeroline=False,  # Remove the zero line
        showticklabels=False

        )
annotationX = dict(x=0.99*limits,y=0,z=0,text="x".format(limits),showarrow=False,font=dict(size=10,color=textcolor))
annotationY = dict(x=0,y=0.99*limits,z=0,text="y".format(limits),showarrow=False,font=dict(size=10,color=textcolor))
annotationZ = dict(x=0,y=0,z=0.99*limits,text="z".format(limits),showarrow=False,font=dict(size=10,color=textcolor))




In [71]:
i0=int(np.abs(time.shape[0] - time_stamps.shape[0]))

In [72]:
i=i0+30
## coordinate the time indexes
time_index_foorb = np.argmin(np.abs(time-time_stamps[i]))
simulation_time_index = np.argmin(np.abs(ts-time_stamps[i]))

In [73]:
my_time_indexing = time_indexing[time_index_foorb]
x_stream,y_stream,z_stream,ax_sub,ay_sub,az_sub,magnitude_sub=prepare_proxy_stream(tau,(xH,yH,zH),(ax[time_index_foorb],ay[time_index_foorb],az[time_index_foorb],magnitude[time_index_foorb]),tau_left[i],tau_right[i],my_time_indexing,tau_indexing)

In [74]:
# leftdex=np.argmin(np.abs(tau - tau_left[i]))
# rightdex=np.argmin(np.abs(tau - tau_right[i]))
# cond1 = blank_dexes >= leftdex
# cond2 = blank_dexes <= rightdex
# cond = np.logical_and(cond1, cond2)
# # extract the stream-proxy data
# x_stream=xH[time_indexing[time_index_foorb] + tau_indexing[cond]]
# y_stream=yH[time_indexing[time_index_foorb] + tau_indexing[cond]]
# z_stream=zH[time_indexing[time_index_foorb] + tau_indexing[cond]]

# # get the force on the stream-proxy
# ax_sub=ax[time_index_foorb][cond]
# ay_sub=ay[time_index_foorb][cond]
# az_sub=az[time_index_foorb][cond]
# magnitude_sub=magnitude[time_index_foorb][cond]
# # normalize the force
# ax_sub=ax_sub/magnitude_sub
# ay_sub=ay_sub/magnitude_sub
# az_sub=az_sub/magnitude_sub

In [75]:
# get the positions of all the perturbers at this time
x_per,y_per,z_per = xs[:,simulation_time_index], ys[:,simulation_time_index],zs[:,simulation_time_index]

In [76]:
# get the position of the host at this time
xHH,yHH,zHH=xH[simulation_time_index],yH[simulation_time_index],zH[simulation_time_index]

In [77]:
# get the current camera angle
mytheta = meanThetaDot*(time_stamps[i]-tH[0]) + thetaH[0]
x_camera = camera_distance*np.cos(mytheta)
y_camera = camera_distance*np.sin(mytheta)
z_camera = (1/2)*camera_distance

In [78]:


def create_traces(host,stream,forcevectors,perturber,sizes,cone_params):
        x_per,y_per,z_per = perturber
        x_stream,y_stream,z_stream = stream
        xHH,yHH,zHH = host
        x_cone,y_cone,z_cone,ax_sub,ay_sub,az_sub = forcevectors
        perturber_trace = go.Scatter3d(x=x_per, y=y_per, z=z_per, mode='markers', marker=dict(size=sizes,color=perturbercolor),showlegend=False)
        host_trace = go.Scatter3d(x=[xHH], y=[yHH], z=[zHH], mode='markers', marker=dict(size=4),showlegend=False)
        stream_trace = go.Scatter3d(x=x_cone, y=y_cone, z=z_cone, mode='lines', marker=dict(size=4),showlegend=False)
        # Add 3D cones for the vectors
        cone=go.Cone(x=x_stream, y=y_stream, z=z_stream,u=ax_sub, v=ay_sub, w=az_sub,**cone_params)
        return perturber_trace,host_trace,stream_trace,cone


def prepare_scene(x_camera,y_camera,z_camera,xaxis,yaxis,zaxis,annotationX,annotationY,annotationZ):
        camera = dict(eye=dict(x=x_camera, y=y_camera, z=z_camera))
        scene=dict(
                camera=camera,
                xaxis=xaxis,
                yaxis=yaxis,
                zaxis=zaxis,
                aspectmode='cube',
                annotations=[annotationX,annotationY,annotationZ]
                )
        return scene

In [98]:
# Define common parameters
cone_params = {
    'colorscale': 'rainbow',
    'cmin': 0,
    'cmax': 50,
    'sizemode': 'absolute',
    'sizeref': 1,
    'showscale': True,
    'colorbar': dict(
        title=dict(text=r'g '),
        thickness=5,  # Adjust thickness
        len=0.5       # Adjust length
    )
}  

In [99]:
np.sqrt(ax_sub**2 + ay_sub**2 + az_sub**2)

array([0.45080576, 0.45179975, 0.45301319, 0.45444736, 0.45610185,
       0.45797513, 0.46006522, 0.46237032])

In [100]:
scene=prepare_scene(x_camera,y_camera,z_camera,xaxis,yaxis,zaxis,annotationX,annotationY,annotationZ)
perturber_trace,host_trace,stream_trace,cone=create_traces((xHH,yHH,zHH),(x_stream,y_stream,z_stream),(x_stream,y_stream,z_stream,ax_sub,ay_sub,az_sub),(x_per,y_per,z_per),sizes,cone_params)

In [101]:
fig = go.Figure()

fig.add_trace(xaxisline)
fig.add_trace(yaxisline)
fig.add_trace(zaxisline)
fig.add_trace(perturber_trace)
fig.add_trace(host_trace)
fig.add_trace(stream_trace)
fig.add_trace(cone) 


fig.update_layout(scene=scene,  
    margin=dict(l=0, r=0, t=0, b=0),  # Reduce margins to make the plot area larger
    scene_aspectmode='manual',  # Set aspect mode to manual
    scene_aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio to make the plot area larger
    plot_bgcolor=backgroundcolor,  # Plot area background color
    paper_bgcolor=backgroundcolor,  # Entire figure background color
)


# fig.write_image("frame-"+str(i).zfill(4)+".png")   # Save as PNG


$$ \frac{km}{s} / \frac{s kpc}{km} $$ 
$$ \frac{km^2}{kpc s^2} $$